In [93]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd

In [94]:
fm_dataset = pd.read_csv("Data/Apache_AllNumeric.csv")
# fm_dataset = fm_dataset.shuffle()
fm_features = fm_dataset.copy()
fm_labels = fm_features.pop('PERF')
fm_labels_max = fm_labels.max()
fm_labels /= fm_labels_max

In [95]:
inputs = {}

for name, column in fm_features.items():
    dtype = tf.float32
    inputs[name] = keras.Input(shape=(1,), name=name, dtype=dtype)

In [96]:
numeric_inputs  = {name: input for name, input in inputs.items()}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(fm_dataset[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

In [97]:
fm_preprocessing = keras.Model(inputs, all_numeric_inputs)
# tf.keras.utils.plot_model(model = fm_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

In [98]:
fm_features_dict = {name: np.array(value) for name, value in fm_features.items()}

In [99]:
def fm_model(preprocessing_head, inputs):
    body = keras.Sequential([
        layers.Dense(20, activation='relu', kernel_regularizer=keras.regularizers.l1(0.009)),
        layers.Dense(10, activation='relu', kernel_regularizer=keras.regularizers.l1(0.009)),
        layers.Dense(1)
    ])
    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = keras.Model(inputs, result)
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                loss="mean_squared_error",
                metrics=[keras.metrics.MeanSquaredError()])
    return model

In [100]:
fm_model = fm_model(fm_preprocessing, inputs)
fm_model.fit(x=fm_features_dict, y=fm_labels, epochs=20, batch_size=10)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


20/20 [==============================] - 1s 2ms/step - loss: 1.4225 - mean_squared_error: 0.7319
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 0.9075 - mean_squared_error: 0.2213
Epoch 3/20
20/20 [==============================] - 0s 3ms/step - loss: 0.7809 - mean_squared_error: 0.1010
Epoch 4/20
20/20 [==============================] - 0s 2ms/step - loss: 0.7511 - mean_squared_error: 0.0806
Epoch 5/20
20/20 [==============================] - 0s 2ms/step - loss: 0.7229 - mean_squared_error: 0.0632
Epoch 6/20
20/20 [==============================] - 0s 2ms/step - loss: 0.7000 - mean_squared_error: 0.0517
Epoch 7/20
20/20 [==============================] - 0s 3ms/step - loss: 0.6798 - mean_squared_error: 0.0434
Epoch 8/20
20/20 [==============================] - 0s 2ms/step - loss: 0.6608 - mean_squared_error: 0.0367
Epoch 9/20
20/20 [==============================] - 0s 3ms/step - loss: 0.6428 - mean_squared_error: 0.0313
Epoch 10/20
20/20 [====================

In [112]:
x=np.array([np.array(value) for value in [0,1,1,0,1,0,1,0]])

fm_model.predict(x=x)

ValueError: in user code:

    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.7/dist-packages/keras/engine/input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expects {len(input_spe